In [1]:
!pip install crewai
!pip install duckduckgo-search
!pip install langchain_community
#export GROQ_API_KEY=<your-api-key-here>

In [2]:
from langchain_community.llms import Ollama
from langchain_community.tools import DuckDuckGoSearchRun
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import os
from langchain_groq import ChatGroq

In [5]:
#api_key= os.environ.get("$GROQ_API_KEY")
#model_name="llama-3.1-70b-versatile"

#llm = ChatGroq(
#    model=model_name,
#    temperature=0.7,
#    max_tokens=1024,
#    timeout=None,
#    max_retries=2
#)

In [6]:
search_tool = DuckDuckGoSearchRun()

In [7]:
os.environ["OPENAI_API_KEY"] = "NA"

llm = ChatOpenAI(
    model = "llama3.1",
    base_url = "http://192.168.1.124:11434/v1",
    temperature=0.7,
    max_tokens=1024,
    timeout=None,
    max_retries=2
)

In [8]:
inputs = {"topic":"Step by step example of installing Apache Ignite on Docker."}

In [9]:
#content planner Agent
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} in "
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "You have to prepare a detailed "
              "outline and the relevant topics and sub-topics that has to be a part of the"
              "blogpost."
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [10]:
#content writer Agent
writer = Agent(
  role='Technical content writer',
  goal="Write detailed and factually accurate "
         "opinion piece about the topic: {topic}",
  backstory="You're working on a writing "
              "a new step by step tutorial about the topic: {topic} in "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [11]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization 'https://www.shamimbhuiyan.ru'. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [12]:
# Create tasks for the planner agent
plan = Task(
    description=(
        "1. Step by step example , "
            "and noteworthy latest news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, values, key points, and step by step example.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    tools=[search_tool],
)

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 3 or 5 paragraphs.",
    agent=writer,
)

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with human language for better readability while maintaining its original meaning."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 3 or 5 paragraphs.",
    agent=editor
)

In [15]:
# Instantiate the crew with a sequential process
crew = Crew(
  agents=[planner, writer, editor],
  tasks=[plan, edit, write],
  verbose=2, # You can set it to 1 or 2 to different logging levels
) 

In [16]:
# run the crew
result = crew.kickoff(inputs=inputs)

print("**Final result**")
print(result)

 [2024-09-26 22:11:50][DEBUG]: == Working Agent: Content Planner
 [2024-09-26 22:11:50][INFO]: == Starting Task: 1. Step by step example , and noteworthy latest news on Step by step example of installing Apache Ignite on Docker..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, values, key points, and step by step example.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by gathering information about installing Apache Ignite on Docker, specifically the latest news and noteworthy updates.

Action: duckduckgo_search
Action Input: {'query': 'latest news Apache Ignite Docker'} 

The new Apache Ignite 2.11.1 is an emergency release that fixes CVE-2021-44228, CVE-2021-45046,CVE-2021-45105 related to the ignite-log4j2 module usage.. Apache Ignite with Log4j Vulnerability. All the following conditions must be met: The Apache Ig